In [1]:
%matplotlib widget
from spectraclass.data.base import DataManager
from spectraclass.data.spatial.tile.manager import TileManager
from spectraclass.application.controller import app, SpectraclassController
from spectraclass.model.labels import LabelsManager, lm
from spectraclass.learn.manager import ClassificationManager, cm
from spectraclass.data.base import ModeDataManager
from spectraclass.data.spatial.modes import AvirisDataManager
from typing import List, Union, Tuple, Optional, Dict, Callable

ImportError: dlopen(/opt/anaconda3/envs/spectraclass1/lib/python3.8/site-packages/rasterio/_base.cpython-38-darwin.so, 2): Library not loaded: @rpath/libfontconfig.1.dylib
  Referenced from: /opt/anaconda3/envs/spectraclass1/lib/libpoppler.117.0.0.dylib
  Reason: Incompatible library version: libpoppler.117.dylib requires version 14.0.0 or later, but libfontconfig.1.dylib provides version 13.0.0

Here we configure paths on the Jupyter server.  If these paths are not specified here then the default values,
    defined in server-side config files, for the project ("demo2") and data mode ("desis"), will be used.  You can
    choose whatever project names you want, they are used to save configurations and results for ongoing investigations.

In [ ]:
dm: DataManager = DataManager.initialize( "demo2", 'aviris' )
dm.modal.cache_dir = "/adapt/nobackup/projects/ilab/cache"
dm.modal.data_dir = "/css/above/AVIRIS_Analysis_Ready"
dm.proc_type = "cpu"
TileManager.block_size = 125
ModeDataManager.model_dims =  16
AvirisDataManager.valid_aviris_bands = [ [5,193], [214,283], [319,10000] ]

Here we load the project data and define a set of class names with associated colors to be used in the classification process.

In [ ]:
dm.loadCurrentProject()
classes = [ ('Class-1', "cyan"),
            ('Class-2', "green"),
            ('Class-3', "magenta"),
            ('Class-4', "blue")]
lm().setLabels( classes )

Add a custom CNN

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras import datasets, layers, models
from spectraclass.learn.models.network import Network, ModelType

class CNN(Network):
    TYPE = ModelType.SPATIAL

    def _build_model(self, **kwargs) -> Tuple[Model,Dict]:
        from spectraclass.learn.models.spatial import SpatialModelWrapper
        nfeatures = kwargs.pop('nfeatures', 32 )
        from spectraclass.model.labels import lm
        input_shape = SpatialModelWrapper.get_input_shape()
        nclasses = lm().nLabels
        ks = kwargs.pop('kernel_size',3)
        model = models.Sequential()
        model.add( Input( shape=input_shape ) )
        model.add( layers.Conv2D( nfeatures, (ks,ks), activation='relu', padding="same" ) )
        model.add( layers.Reshape( SpatialModelWrapper.flatten(input_shape,nfeatures) ) )
        model.add( layers.Dense( nfeatures, activation='relu' ) )
 #       model.add( layers.Dense( nfeatures, activation='relu') )
        model.add( layers.Dense( nclasses, activation='softmax' ) )
        return model, kwargs

cm().addNetwork( CNN('cnn-1', nfeatures=64 ) )

Here we start up the Spectraclass GUI.

In [ ]:
controller: SpectraclassController = app()
controller.gui()